In [1]:
import sys
import matplotlib.pyplot as plt
import numpy as np
import pickle
import tensorflow as tf
import glob
from sklearn.preprocessing import MinMaxScaler
from __future__ import print_function, division
from music21 import converter, instrument, note, chord, stream
from keras.layers import Input, Dense, Reshape, Dropout, CuDNNLSTM, Bidirectional,LSTM,Embedding
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.utils import np_utils

# Check that we are using a GPU, if not switch runtimes
#   using Runtime > Change Runtime Type > GPU
#assert len(tf.config.list_physical_devices('GPU')) > 0

In [26]:

import csv
music_data = []
with open("/content/drive/MyDrive/Thesis/tunes.csv", "r") as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for lines in csv_reader:
      music_data.append(lines[1])


In [27]:
joinedsongs = []
for i in music_data:
  tune = "\n".join(i)
  joinedsongs.append(tune)
  
final_tunes = "\n\n".join(joinedsongs) 

In [31]:
# Find all unique characters in the joined string
vocab = sorted(set(final_tunes))
print("There are", len(vocab), "unique characters in the dataset")

There are 122 unique characters in the dataset


In [32]:
def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    vocab = sorted(set(notes))
    sequence_length = len(vocab)

    
    char2int = {u:i for i, u in enumerate(vocab)}
    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([char2int[char] for char in sequence_in])
        network_output.append(char2int[sequence_out])

    n_patterns = len(network_input)

    # Reshape the input into a format compatible with LSTM layers
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))


    return (network_input, network_output)

In [ ]:
x,y = prepare_sequences(final_tunes,len(set(final_tunes)))

In [ ]:
class GAN():
    def __init__(self, rows):
        self.seq_length = rows
        self.seq_shape = (self.seq_length, 1)
        self.latent_dim = 100
        self.disc_loss = []
        self.gen_loss =[]
        
        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
        
        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates note sequences
        z = Input(shape=(1,self.latent_dim))
        generated_seq = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        validity = self.discriminator(generated_seq)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def lstm_gen(rnn_units): 
        return tf.keras.layers.LSTM(
            rnn_units, 
            return_sequences=True, 
            recurrent_initializer='glorot_uniform',
            recurrent_activation='sigmoid',
            stateful=True, 
            )
    
    def build_discriminator(self):
        model = Sequential()
        model.add(LSTM(512,return_sequences=True,input_shape = self.seq_shape))
        model.add(Bidirectional(LSTM(512)))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()
        seq = Input(shape=self.seq_shape)
        validity = model(seq)

        return Model(seq, validity)                

    def build_generator(self):
        model = Sequential()
        model.add(LSTM(256,input_shape = (1,self.latent_dim),return_sequences=True))
        model.add(Dropout(0.6))
        model.add(LSTM(128,input_shape = (1,self.latent_dim),return_sequences=True))
        model.add(Dropout(0.6))
        model.add(LSTM(64,input_shape = (1,self.latent_dim),return_sequences=False))
        model.add(Dropout(0.6))
        model.add(Dense(np.prod(self.seq_shape), activation='tanh'))
        model.add(Reshape(self.seq_shape))
        model.summary()
        
        noise = Input(shape=(1,self.latent_dim))
        seq = model(noise)

        return Model(noise, seq)

    def train(self,final_tunes, epochs,gen_len, batch_size=32, sample_interval=50):

        # Load and convert the data
        notes = final_tunes
        n_vocab = len(set(final_tunes))
        X_train, y_train = prepare_sequences(notes, n_vocab)

        # Adversarial ground truths
        real = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        
        # Training the model
        for epoch in range(epochs):

            # Training the discriminator
            # Select a random batch of note sequences
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            real_seqs = X_train[idx]

            #noise = np.random.choice(range(484), (batch_size, self.latent_dim))
            #noise = (noise-242)/242
            noise = np.random.normal(0, 1, (batch_size, 1,self.latent_dim))
            gen_seqs = self.generator.predict(noise)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(real_seqs, real)
            d_loss_fake = self.discriminator.train_on_batch(gen_seqs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)


            #  Training the Generator
            noise = np.random.normal(0, 1, (batch_size, 1,self.latent_dim))
            # Train the generator (to have the discriminator label samples as real)
            g_loss = self.combined.train_on_batch(noise, real)

            # Print the progress and save into loss lists
            if epoch % sample_interval == 0:
              print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
              self.disc_loss.append(d_loss[0])
              self.gen_loss.append(g_loss)
        
        op = self.generate(notes,gen_len)
        self.plot_loss()
        return op

    def generate(self, input_notes,gen_len):
        # Get pitch names and store in a dictionary
        notes = input_notes
        final = []
        pitchnames = sorted(set(item for item in notes))
        idx2char = dict((number, note) for number, note in enumerate(pitchnames))
        # Use random noise to generate sequences
        for i in range(gen_len):
          noise = np.random.normal(0, 1, (1,1, self.latent_dim))
          predictions = self.generator.predict(noise)
          #print('pred:',predictions)
          predictions = tf.squeeze(predictions, 0)
          predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
          print('id:',predicted_id)
          #final.append(idx2char[predicted_id])


    def plot_loss(self):
        plt.plot(self.disc_loss, c='red')
        plt.plot(self.gen_loss, c='blue')
        plt.title("GAN Loss per Epoch")
        plt.legend(['Discriminator', 'Generator'])
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.savefig('GAN_Loss_per_Epoch_final.png', transparent=True)
        plt.close()

if __name__ == '__main__':
  gan = GAN(rows=len(vocab))    
  op = gan.train(final_tunes,epochs=1000,gen_len=20, batch_size=32, sample_interval=1)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 122, 512)          1052672   
_________________________________________________________________
bidirectional (Bidirectional (None, 1024)              4198400   
_________________________________________________________________
dense (Dense)                (None, 512)               524800    
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 2

In [ ]:
predictions = tf.squeeze(op, 0)

In [ ]:
predicted_id = tf.random.categorical(op[0], num_samples=1)